# Taleb Barbell: Massive Parameter Sweep

## Sell ATM Volatility + Buy Deep OTM Puts — ~924 Configurations on SPY 2008-2025

From Taleb's [2015 Reddit AMA](https://www.reddit.com/r/options/comments/38onec/):

> *"I am not against selling ATM premium. This is one of the nonsense people have spread in the interpretation of my ideas."*

> *"ATM drops faster, OTM rises. The same idea of shadow theta."*

> *"For squeezes, a six month OTM (measuring in low delta) is preferable to a 1 year OTM with higher delta."*

### The Strategy

**Pure options, no stock allocation.** The portfolio is 100% deployed into two option legs:

| Leg | Action | Why |
|-----|--------|-----|
| **ATM Straddle** | Sell ATM call + put (strike ± 0.5–2% of spot) | Harvest Variance Risk Premium at its densest point |
| **Deep OTM Put** | Buy far-OTM puts (delta < 0.10) | Convex tail insurance — explosive gamma in crashes |

### Margin Constraint: `max_notional_pct`

Short option premium is small relative to notional risk (strike × 100 × qty). A 1% options allocation can create 10-12x leverage. During flash crashes, this causes the portfolio to go deeply negative — impossible in reality where brokers enforce margin.

`max_notional_pct` caps total short option notional at a percentage of portfolio value, mimicking real-world margin constraints. Default: **30%** (Phases 1-2), swept at **20%, 30%, 50%** (Phase 3).

### Sweep Design (~924 configs)

| Phase | What | Configs |
|-------|------|--------|
| 1 | ATM straddle params (81 combos × 2 allocs) with fixed OTM defaults, 30% notional cap | 162 |
| 2 | OTM put params (81 combos × 2 allocs) with fixed ATM defaults, 30% notional cap | 162 |
| 3 | Top 5 ATM × Top 5 OTM × 4 budget splits × 2 rebalance freqs × 3 notional caps | 600 |
| **Total** | | **924** |

### Crisis Periods Analyzed (7)

| Event | Dates | SPY Drop |
|-------|-------|----------|
| 2008 GFC | Oct 2007 – Mar 2009 | -56% |
| 2011 US Downgrade | Jul – Oct 2011 | -19% |
| 2015 China Deval | Aug 10–25, 2015 | -12% |
| 2018 Volmageddon | Jan 26 – Feb 8, 2018 | -10% |
| 2018 Q4 Selloff | Oct – Dec 2018 | -20% |
| 2020 COVID | Feb – Mar 2020 | -34% |
| 2022 Bear Market | Jan – Oct 2022 | -25% |

### References

- Taleb, N.N. *Dynamic Hedging* (1997)
- Carr & Wu, *Variance Risk Premia* (2009)
- Spitznagel, M. *Safe Haven* (2021)
- Ilmanen & Israelov, *Tail Risk Hedging* (AQR, 2018)

In [ ]:
import os, sys, math, time, warnings
warnings.filterwarnings('ignore')

from concurrent.futures import ProcessPoolExecutor
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

PROJECT_ROOT = os.path.realpath(os.path.join(os.getcwd(), '..'))
NOTEBOOKS_DIR = os.path.dirname(os.path.abspath('taleb_barbell.ipynb'))
# If running via nbconvert, cwd may not be notebooks/
if not os.path.exists('nb_style.py'):
    NOTEBOOKS_DIR = os.path.join(PROJECT_ROOT, 'notebooks')
sys.path.insert(0, PROJECT_ROOT)
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'scripts'))
sys.path.insert(0, NOTEBOOKS_DIR)
os.chdir(PROJECT_ROOT)

from options_portfolio_backtester import BacktestEngine, Stock, Direction
from options_portfolio_backtester.core.types import OptionType as Type
from options_portfolio_backtester.data.providers import HistoricalOptionsData, TiingoData
from options_portfolio_backtester.strategy.strategy import Strategy
from options_portfolio_backtester.strategy.strategy_leg import StrategyLeg
from options_portfolio_backtester.analytics.stats import BacktestStats
from nb_style import (apply_style, shade_crashes, style_returns_table,
                       FT_GREEN, FT_RED, FT_BLUE, FT_DARK, FT_BG, FT_GREY,
                       color_excess)

apply_style()
%matplotlib inline

INITIAL_CAPITAL = 1_000_000
# Each worker loads ~12GB peak memory. Cap to 3 on 48GB to avoid OOM.
N_CORES = min(os.cpu_count(), 3)

# 7 crisis periods
CRISES = [
    ('2008 GFC',          '2007-10-01', '2009-03-09'),
    ('2011 US Downgrade',  '2011-07-22', '2011-10-03'),
    ('2015 China Deval',   '2015-08-10', '2015-08-25'),
    ('2018 Volmageddon',   '2018-01-26', '2018-02-08'),
    ('2018 Q4 Selloff',    '2018-10-01', '2018-12-24'),
    ('2020 COVID',         '2020-02-19', '2020-03-23'),
    ('2022 Bear',          '2022-01-03', '2022-10-12'),
]

# Palette for top configs
TOP_COLORS = [FT_BLUE, FT_RED, FT_GREEN, '#E68A00', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

print(f'Cores: {N_CORES} | Ready.')

In [ ]:
options_data = HistoricalOptionsData('data/processed/options.csv')
stocks_data = TiingoData('data/processed/stocks.csv')
schema = options_data.schema

spy_prices = (
    stocks_data._data[stocks_data._data['symbol'] == 'SPY']
    .set_index('date')['adjClose']
    .sort_index()
)
years = (spy_prices.index[-1] - spy_prices.index[0]).days / 365.25
spy_total_ret = (spy_prices.iloc[-1] / spy_prices.iloc[0] - 1) * 100
spy_annual_ret = ((1 + spy_total_ret / 100) ** (1 / years) - 1) * 100
spy_cummax = spy_prices.cummax()
spy_dd = ((spy_prices - spy_cummax) / spy_cummax).min() * 100

print(f'Date range: {stocks_data.start_date} to {stocks_data.end_date} ({years:.1f} years)')
print(f'SPY B&H: {spy_total_ret:.1f}% total, {spy_annual_ret:.2f}%/yr, {spy_dd:.1f}% max DD')

In [ ]:
def make_barbell_strategy(
    schema,
    # ATM straddle (sell) params
    atm_dte_min=45, atm_dte_max=120, atm_exit_dte=14,
    atm_strike_width=0.01,
    # Deep OTM put (buy) params
    otm_delta_min=-0.10, otm_delta_max=-0.02,
    otm_dte_min=90, otm_dte_max=180, otm_exit_dte=14,
):
    """Build a Taleb barbell: sell ATM straddle + buy deep OTM put.
    
    Returns a Strategy with 3 legs:
      leg_1: sell ATM call
      leg_2: sell ATM put  
      leg_3: buy deep OTM put
    """
    lo = 1.0 - atm_strike_width
    hi = 1.0 + atm_strike_width

    # Leg 1: Sell ATM call
    atm_call = StrategyLeg('leg_1', schema, option_type=Type.CALL, direction=Direction.SELL)
    atm_call.entry_filter = (
        (schema.underlying == 'SPY')
        & (schema.dte >= atm_dte_min) & (schema.dte <= atm_dte_max)
        & (schema.strike >= schema.underlying_last * lo)
        & (schema.strike <= schema.underlying_last * hi)
    )
    atm_call.entry_sort = ('delta', False)  # closest to 0.50
    atm_call.exit_filter = schema.dte <= atm_exit_dte

    # Leg 2: Sell ATM put
    atm_put = StrategyLeg('leg_2', schema, option_type=Type.PUT, direction=Direction.SELL)
    atm_put.entry_filter = (
        (schema.underlying == 'SPY')
        & (schema.dte >= atm_dte_min) & (schema.dte <= atm_dte_max)
        & (schema.strike >= schema.underlying_last * lo)
        & (schema.strike <= schema.underlying_last * hi)
    )
    atm_put.entry_sort = ('delta', True)  # closest to -0.50
    atm_put.exit_filter = schema.dte <= atm_exit_dte

    # Leg 3: Buy deep OTM put (tail hedge)
    otm_put = StrategyLeg('leg_3', schema, option_type=Type.PUT, direction=Direction.BUY)
    otm_put.entry_filter = (
        (schema.underlying == 'SPY')
        & (schema.dte >= otm_dte_min) & (schema.dte <= otm_dte_max)
        & (schema.delta >= otm_delta_min) & (schema.delta <= otm_delta_max)
    )
    otm_put.entry_sort = ('delta', False)  # deepest OTM first
    otm_put.exit_filter = schema.dte <= otm_exit_dte

    s = Strategy(schema)
    s.add_legs([atm_call, atm_put, otm_put])
    s.add_exit_thresholds(profit_pct=math.inf, loss_pct=math.inf)
    return s


print('Strategy builder ready.')

In [ ]:
from barbell_worker import run_config

print('Worker function loaded from barbell_worker.py')

In [ ]:
# ── ATM straddle grid (81 combos) ─────────────────────────────────
ATM_DTE_MINS   = [30, 45, 60]
ATM_DTE_MAXS   = [90, 120, 150]
ATM_EXIT_DTES  = [7, 14, 30]
ATM_WIDTHS     = [0.005, 0.01, 0.02]   # ±0.5%, ±1%, ±2% of spot

# ── OTM put grid (81 combos) ──────────────────────────────────────
OTM_DELTAS     = [(-0.10, -0.02), (-0.15, -0.05), (-0.08, -0.01)]
OTM_DTE_MINS   = [60, 90, 120]
OTM_DTE_MAXS   = [150, 180, 240]
OTM_EXIT_DTES  = [7, 14, 30]

# ── Allocation / rebalance ────────────────────────────────────────
OPT_PCTS       = [0.01, 0.02, 0.03, 0.05]  # 1%, 2%, 3%, 5%
REBAL_FREQS    = [1, 3]                      # monthly, quarterly

# ── Notional cap ──────────────────────────────────────────────────
# max_notional_pct caps total short option notional at a % of portfolio
# value, mimicking real-world margin constraints. Without it, selling
# straddles creates 10-12x leverage that blows up during crashes.
DEFAULT_NOTIONAL_CAP = 0.30   # 30% of portfolio — ~1-3 SPY straddles
NOTIONAL_CAPS  = [0.20, 0.30, 0.50]  # swept in Phase 3

# ── Fixed defaults (used when sweeping only one side) ─────────────
ATM_DEFAULTS = dict(atm_dte_min=45, atm_dte_max=120, atm_exit_dte=14, atm_strike_width=0.01)
OTM_DEFAULTS = dict(otm_delta_min=-0.10, otm_delta_max=-0.02, otm_dte_min=90, otm_dte_max=180, otm_exit_dte=14)

print(f'ATM combos: {len(ATM_DTE_MINS)*len(ATM_DTE_MAXS)*len(ATM_EXIT_DTES)*len(ATM_WIDTHS)} = 81')
print(f'OTM combos: {len(OTM_DELTAS)*len(OTM_DTE_MINS)*len(OTM_DTE_MAXS)*len(OTM_EXIT_DTES)} = 81')
print(f'Allocs (options %): {OPT_PCTS}, Rebal freqs: {REBAL_FREQS}')
print(f'Default notional cap: {DEFAULT_NOTIONAL_CAP*100:.0f}% of portfolio')
print(f'Notional caps swept in Phase 3: {[f"{c*100:.0f}%" for c in NOTIONAL_CAPS]}')

---
## Quick Smoke Test — Capped vs Uncapped

Run one barbell config with default params. Compare **uncapped** (old behavior, unrealistic leverage) vs **capped at 30%** (realistic margin constraint).

In [ ]:
# Smoke test: default params, 1% options / 99% SPY, monthly rebal
# Compare uncapped (old behavior) vs capped (realistic margin)
base_args = (
    60, 120, 30, 0.01,       # ATM: dte_min, dte_max, exit_dte, strike_width
    -0.10, -0.02,            # OTM: delta_min, delta_max
    90, 180, 14,             # OTM: dte_min, dte_max, exit_dte
)

t0 = time.perf_counter()
result_uncapped = run_config(('uncapped', 0.01, 1, *base_args, None))
result_capped   = run_config(('capped_30pct', 0.01, 1, *base_args, DEFAULT_NOTIONAL_CAP))
elapsed = time.perf_counter() - t0

print(f'Time: {elapsed:.1f}s  |  Allocation: 99% SPY + 1% options\n')
print(f'{"":20s}  {"Uncapped":>12s}  {"Capped (30%)":>12s}')
print('-' * 50)
print(f'{"Annual return":20s}  {result_uncapped["annual_ret"]:>11.2f}%  {result_capped["annual_ret"]:>11.2f}%')
print(f'{"Max drawdown":20s}  {result_uncapped["max_dd"]:>11.1f}%  {result_capped["max_dd"]:>11.1f}%')
print(f'{"Volatility":20s}  {result_uncapped["vol"]:>11.1f}%  {result_capped["vol"]:>11.1f}%')
print(f'{"Sharpe":20s}  {result_uncapped["sharpe"]:>11.3f}   {result_capped["sharpe"]:>11.3f}')
print(f'{"Trades":20s}  {result_uncapped["trades"]:>11d}   {result_capped["trades"]:>11d}')
print(f'\nSPY B&H: {spy_annual_ret:.2f}%/yr, {spy_dd:.1f}% max DD')

result = result_capped  # use capped for the capital curve

print(f'\nCrisis returns (capped):')
for label in ['2008 GFC', '2011 US Downgrade', '2020 COVID', '2022 Bear']:
    uc = result_uncapped.get(label, float('nan'))
    ca = result_capped.get(label, float('nan'))
    print(f'  {label}: {ca:.1f}%  (uncapped: {uc:.1f}%)')

# Capital curves: capped vs uncapped vs SPY
fig, ax = plt.subplots(figsize=(16, 5))
spy_norm = spy_prices / spy_prices.iloc[0] * INITIAL_CAPITAL
ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2, alpha=0.7, label='SPY B&H')

for r, c, lbl in [(result_uncapped, FT_RED, 'Uncapped'), (result_capped, FT_BLUE, f'Capped {DEFAULT_NOTIONAL_CAP*100:.0f}%')]:
    bal = pd.Series(r['balance_values'])
    bal.index = pd.to_datetime(bal.index)
    ax.plot(bal.index, bal.values, color=c, lw=2, label=f'{lbl} ({r["annual_ret"]:.1f}%/yr, DD {r["max_dd"]:.0f}%)')

shade_crashes(ax)
ax.set_title('Smoke Test: Notional Cap Effect on Barbell (99% SPY + 1% vol overlay)', fontsize=14)
ax.set_ylabel('Portfolio Value ($)')
ax.ticklabel_format(style='plain', axis='y')
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

---
## Phase 1: ATM Straddle Sweep

Sweep 81 ATM parameter combos × 2 allocations (2%, 5% options) = **162 configs**.
OTM put uses fixed defaults. Rest of capital in SPY equity.

In [ ]:
phase1_configs = []
for atm_min, atm_max, atm_exit, width in product(ATM_DTE_MINS, ATM_DTE_MAXS, ATM_EXIT_DTES, ATM_WIDTHS):
    for opt_pct in [0.02, 0.05]:
        name = f'ATM:{atm_min}-{atm_max}/ex{atm_exit}/w{width:.3f} | opt{opt_pct*100:.0f}%'
        phase1_configs.append((
            name, opt_pct, 1,  # monthly rebal
            atm_min, atm_max, atm_exit, width,
            OTM_DEFAULTS['otm_delta_min'], OTM_DEFAULTS['otm_delta_max'],
            OTM_DEFAULTS['otm_dte_min'], OTM_DEFAULTS['otm_dte_max'],
            OTM_DEFAULTS['otm_exit_dte'],
            DEFAULT_NOTIONAL_CAP,
        ))

print(f'Phase 1: {len(phase1_configs)} configs on {N_CORES} cores (notional cap: {DEFAULT_NOTIONAL_CAP*100:.0f}%)')

t0 = time.perf_counter()
# Time one config first
one_result = run_config(phase1_configs[0])
t1 = time.perf_counter() - t0
print(f'One config: {t1:.1f}s → est parallel: {t1 * len(phase1_configs) / N_CORES:.0f}s')

t0 = time.perf_counter()
with ProcessPoolExecutor(max_workers=N_CORES) as ex:
    phase1_results = list(ex.map(run_config, phase1_configs))
elapsed = time.perf_counter() - t0
print(f'Phase 1 done: {elapsed:.1f}s ({elapsed/60:.1f}min)')

phase1_results.sort(key=lambda r: r['sharpe'], reverse=True)

print(f'\n{"Config":<55} {"Annual":>8} {"MaxDD":>8} {"Vol":>8} {"Sharpe":>8}')
print('-' * 95)
for r in phase1_results[:10]:
    print(f'{r["name"]:<55} {r["annual_ret"]:>7.2f}% {r["max_dd"]:>7.1f}% {r["vol"]:>7.1f}% {r["sharpe"]:>8.3f}')

---
## Phase 2: OTM Put Sweep

Sweep 81 OTM parameter combos × 2 allocations (2%, 5% options) = **162 configs**.
ATM straddle uses fixed defaults. Rest of capital in SPY equity.

In [ ]:
phase2_configs = []
for (d_min, d_max), otm_min, otm_max, otm_exit in product(OTM_DELTAS, OTM_DTE_MINS, OTM_DTE_MAXS, OTM_EXIT_DTES):
    for opt_pct in [0.02, 0.05]:
        name = f'OTM:d({d_min},{d_max})/{otm_min}-{otm_max}/ex{otm_exit} | opt{opt_pct*100:.0f}%'
        phase2_configs.append((
            name, opt_pct, 1,  # monthly rebal
            ATM_DEFAULTS['atm_dte_min'], ATM_DEFAULTS['atm_dte_max'],
            ATM_DEFAULTS['atm_exit_dte'], ATM_DEFAULTS['atm_strike_width'],
            d_min, d_max, otm_min, otm_max, otm_exit,
            DEFAULT_NOTIONAL_CAP,
        ))

print(f'Phase 2: {len(phase2_configs)} configs on {N_CORES} cores (notional cap: {DEFAULT_NOTIONAL_CAP*100:.0f}%)')

t0 = time.perf_counter()
with ProcessPoolExecutor(max_workers=N_CORES) as ex:
    phase2_results = list(ex.map(run_config, phase2_configs))
elapsed = time.perf_counter() - t0
print(f'Phase 2 done: {elapsed:.1f}s ({elapsed/60:.1f}min)')

phase2_results.sort(key=lambda r: r['sharpe'], reverse=True)

print(f'\n{"Config":<60} {"Annual":>8} {"MaxDD":>8} {"Vol":>8} {"Sharpe":>8}')
print('-' * 100)
for r in phase2_results[:10]:
    print(f'{r["name"]:<60} {r["annual_ret"]:>7.2f}% {r["max_dd"]:>7.1f}% {r["vol"]:>7.1f}% {r["sharpe"]:>8.3f}')

In [ ]:
# ATM heatmap: Sharpe by (DTE window × exit DTE)
p1_df = pd.DataFrame(phase1_results)

# Average Sharpe across widths and allocs for each DTE combo
p1_df['dte_window'] = p1_df['atm_dte_min'].astype(str) + '-' + p1_df['atm_dte_max'].astype(str)
hm1 = p1_df.pivot_table(values='sharpe', index='dte_window', columns='atm_exit_dte', aggfunc='mean')
hm1.columns = [f'Exit {c}d' for c in hm1.columns]

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

sns.heatmap(hm1, annot=True, fmt='.3f', cmap='RdYlGn', center=0, ax=axes[0],
            linewidths=0.5, cbar_kws={'label': 'Sharpe'})
axes[0].set_title('ATM Straddle: Sharpe by DTE Window × Exit DTE')
axes[0].set_ylabel('DTE Window (min-max)')

# Sharpe by strike width
hm2 = p1_df.pivot_table(values='sharpe', index='atm_strike_width', columns='atm_exit_dte', aggfunc='mean')
hm2.index = [f'±{w*100:.1f}%' for w in hm2.index]
hm2.columns = [f'Exit {c}d' for c in hm2.columns]

sns.heatmap(hm2, annot=True, fmt='.3f', cmap='RdYlGn', center=0, ax=axes[1],
            linewidths=0.5, cbar_kws={'label': 'Sharpe'})
axes[1].set_title('ATM Straddle: Sharpe by Strike Width × Exit DTE')
axes[1].set_ylabel('Strike Width (% of spot)')

plt.tight_layout()
plt.show()

In [ ]:
# OTM heatmap: Sharpe by (delta range × DTE window)
p2_df = pd.DataFrame(phase2_results)

p2_df['delta_range'] = p2_df['otm_delta_min'].round(2).astype(str) + ' to ' + p2_df['otm_delta_max'].round(2).astype(str)
p2_df['dte_window'] = p2_df['otm_dte_min'].astype(str) + '-' + p2_df['otm_dte_max'].astype(str)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

hm3 = p2_df.pivot_table(values='sharpe', index='delta_range', columns='dte_window', aggfunc='mean')
sns.heatmap(hm3, annot=True, fmt='.3f', cmap='RdYlGn', center=0, ax=axes[0],
            linewidths=0.5, cbar_kws={'label': 'Sharpe'})
axes[0].set_title('OTM Put: Sharpe by Delta Range × DTE Window')
axes[0].set_ylabel('Delta Range')

hm4 = p2_df.pivot_table(values='sharpe', index='delta_range', columns='otm_exit_dte', aggfunc='mean')
hm4.columns = [f'Exit {c}d' for c in hm4.columns]
sns.heatmap(hm4, annot=True, fmt='.3f', cmap='RdYlGn', center=0, ax=axes[1],
            linewidths=0.5, cbar_kws={'label': 'Sharpe'})
axes[1].set_title('OTM Put: Sharpe by Delta Range × Exit DTE')
axes[1].set_ylabel('Delta Range')

plt.tight_layout()
plt.show()

---
## Phase 3: Combined Sweep

Take top 5 ATM configs × top 5 OTM configs × 4 allocations × 2 rebalance frequencies × 3 notional caps = **600 configs**.

In [ ]:
# Extract top 5 unique ATM and OTM param sets
def unique_atm_params(results, n=5):
    seen, out = set(), []
    for r in results:
        key = (r['atm_dte_min'], r['atm_dte_max'], r['atm_exit_dte'], r['atm_strike_width'])
        if key not in seen:
            seen.add(key)
            out.append(dict(atm_dte_min=key[0], atm_dte_max=key[1],
                            atm_exit_dte=key[2], atm_strike_width=key[3]))
        if len(out) >= n:
            break
    return out

def unique_otm_params(results, n=5):
    seen, out = set(), []
    for r in results:
        key = (r['otm_delta_min'], r['otm_delta_max'], r['otm_dte_min'], r['otm_dte_max'], r['otm_exit_dte'])
        if key not in seen:
            seen.add(key)
            out.append(dict(otm_delta_min=key[0], otm_delta_max=key[1],
                            otm_dte_min=key[2], otm_dte_max=key[3], otm_exit_dte=key[4]))
        if len(out) >= n:
            break
    return out

top_atm = unique_atm_params(phase1_results)
top_otm = unique_otm_params(phase2_results)

print('Top 5 ATM configs:')
for i, a in enumerate(top_atm, 1):
    print(f'  {i}. DTE {a["atm_dte_min"]}-{a["atm_dte_max"]}, exit {a["atm_exit_dte"]}d, width ±{a["atm_strike_width"]*100:.1f}%')
print('Top 5 OTM configs:')
for i, o in enumerate(top_otm, 1):
    print(f'  {i}. delta ({o["otm_delta_min"]},{o["otm_delta_max"]}), DTE {o["otm_dte_min"]}-{o["otm_dte_max"]}, exit {o["otm_exit_dte"]}d')

# Build combined grid — now includes notional cap sweep
phase3_configs = []
for i_atm, atm in enumerate(top_atm):
    for i_otm, otm in enumerate(top_otm):
        for opt_pct in OPT_PCTS:
            for rebal in REBAL_FREQS:
                for ncap in NOTIONAL_CAPS:
                    name = (f'A{i_atm+1}×O{i_otm+1} | opt{opt_pct*100:.0f}% | '
                            f'{"M" if rebal == 1 else "Q"} | cap{ncap*100:.0f}%')
                    phase3_configs.append((
                        name, opt_pct, rebal,
                        atm['atm_dte_min'], atm['atm_dte_max'],
                        atm['atm_exit_dte'], atm['atm_strike_width'],
                        otm['otm_delta_min'], otm['otm_delta_max'],
                        otm['otm_dte_min'], otm['otm_dte_max'],
                        otm['otm_exit_dte'],
                        ncap,
                    ))

print(f'\nPhase 3: {len(phase3_configs)} configs on {N_CORES} cores')

t0 = time.perf_counter()
with ProcessPoolExecutor(max_workers=N_CORES) as ex:
    phase3_results = list(ex.map(run_config, phase3_configs))
elapsed = time.perf_counter() - t0
print(f'Phase 3 done: {elapsed:.1f}s ({elapsed/60:.1f}min)')

phase3_results.sort(key=lambda r: r['sharpe'], reverse=True)

print(f'\n{"Config":<40} {"Annual":>8} {"MaxDD":>8} {"Vol":>8} {"Sharpe":>8}')
print('-' * 80)
for r in phase3_results[:10]:
    print(f'{r["name"]:<40} {r["annual_ret"]:>7.2f}% {r["max_dd"]:>7.1f}% {r["vol"]:>7.1f}% {r["sharpe"]:>8.3f}')

---
## Top 20 Combined Results

In [ ]:
# Combine all results for the master table
all_results = phase1_results + phase2_results + phase3_results
all_results.sort(key=lambda r: r['sharpe'], reverse=True)

rows = []
for r in all_results[:20]:
    cap = r.get('max_notional_pct')
    rows.append({
        'Config': r['name'],
        'Opt %': f"{r['opt_pct']*100:.0f}%",
        'Cap %': f"{cap*100:.0f}%" if cap else 'None',
        'Annual %': r['annual_ret'],
        'Max DD %': r['max_dd'],
        'Vol %': r['vol'],
        'Sharpe': r['sharpe'],
        'Trades': r['trades'],
    })

top20_df = pd.DataFrame(rows)
styled = (top20_df.style
    .format({
        'Annual %': '{:.2f}', 'Max DD %': '{:.1f}',
        'Vol %': '{:.1f}', 'Sharpe': '{:.3f}', 'Trades': '{:.0f}',
    })
    .background_gradient(subset=['Sharpe'], cmap='RdYlGn')
    .background_gradient(subset=['Max DD %'], cmap='RdYlGn_r')
)
style_returns_table(styled).set_caption(
    f'Top 20 Barbell Configs by Sharpe  |  SPY B&H: {spy_annual_ret:.2f}%/yr, {spy_dd:.1f}% max DD'
)

---
## Capital Curves — Top 5 vs SPY B&H

In [ ]:
top5 = all_results[:5]

fig, ax = plt.subplots(figsize=(18, 7))

spy_norm = spy_prices / spy_prices.iloc[0] * INITIAL_CAPITAL
ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2.5, alpha=0.7, label='SPY B&H')

for r, c in zip(top5, TOP_COLORS):
    bal = pd.Series(r['balance_values'])
    bal.index = pd.to_datetime(bal.index)
    ax.plot(bal.index, bal.values, color=c, lw=1.8, alpha=0.85,
            label=f"{r['name']} (Sharpe {r['sharpe']:.3f})")

# Shade all 7 crises
crisis_colors = ['#CC0000', '#FF6600', '#CC9900', '#996633', '#663399', '#FF8833', '#9467bd']
for (label, start, end), cc in zip(CRISES, crisis_colors):
    ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.10, color=cc, label=label)

ax.set_title('Taleb Barbell: Top 5 Configs vs SPY Buy & Hold', fontsize=14)
ax.set_ylabel('Portfolio Value ($)')
ax.ticklabel_format(style='plain', axis='y')
ax.legend(fontsize=7, loc='upper left', ncol=2)
plt.tight_layout()
plt.show()

---
## Drawdown Chart — Top 5 vs SPY

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))

spy_dd_s = ((spy_prices - spy_cummax) / spy_cummax * 100)
ax.plot(spy_dd_s.index, spy_dd_s.values, 'k--', lw=2, alpha=0.7, label='SPY B&H')

for r, c in zip(top5, TOP_COLORS):
    bal = pd.Series(r['balance_values'])
    bal.index = pd.to_datetime(bal.index)
    cm = bal.cummax()
    dd = ((bal - cm) / cm * 100)
    ax.plot(dd.index, dd.values, color=c, lw=1.5, alpha=0.8, label=r['name'])

for (label, start, end), cc in zip(CRISES, crisis_colors):
    ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.10, color=cc)

ax.set_title('Drawdowns: Top 5 Barbell Configs vs SPY', fontsize=14)
ax.set_ylabel('% from Peak')
ax.legend(fontsize=7, loc='lower left')
plt.tight_layout()
plt.show()

---
## Risk / Return Scatter — All Configs

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# All configs as grey cloud
all_dd = [abs(r['max_dd']) for r in all_results]
all_ret = [r['annual_ret'] for r in all_results]
ax.scatter(all_dd, all_ret, color=FT_GREY, alpha=0.3, s=20, label=f'All configs (n={len(all_results)})')

# Top 5 highlighted
for r, c in zip(top5, TOP_COLORS):
    ax.scatter(abs(r['max_dd']), r['annual_ret'],
               color=c, s=120, zorder=3, edgecolors='white', linewidths=1.5)
    ax.annotate(r['name'], (abs(r['max_dd']), r['annual_ret']),
                textcoords='offset points', xytext=(8, 5), fontsize=7)

# SPY reference
spy_max_dd_abs = abs(spy_dd)
ax.scatter(spy_max_dd_abs, spy_annual_ret, color='black', s=150, marker='D', zorder=4)
ax.annotate('SPY B&H', (spy_max_dd_abs, spy_annual_ret),
            textcoords='offset points', xytext=(8, 5), fontsize=9, fontweight='bold')

ax.set_xlabel('Max Drawdown (%, absolute)')
ax.set_ylabel('Annual Return (%)')
ax.set_title('Risk / Return: All Barbell Configs', fontsize=14)
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

---
## Crisis Returns — 7 Periods × Top 10 Configs

In [ ]:
top10 = all_results[:10]

# SPY crisis returns
crisis_rows = []
for label, start, end in CRISES:
    row = {'Crisis': label}
    sl = spy_prices[(spy_prices.index >= start) & (spy_prices.index <= end)]
    row['SPY B&H'] = (sl.iloc[-1] / sl.iloc[0] - 1) * 100 if len(sl) > 1 else float('nan')
    for r in top10:
        row[r['name']] = r.get(label, float('nan'))
    crisis_rows.append(row)

crisis_df = pd.DataFrame(crisis_rows).set_index('Crisis')

def color_crisis(val):
    if isinstance(val, (int, float)):
        if val > 0:
            return f'color: {FT_GREEN}; font-weight: bold'
        if val < -10:
            return f'color: {FT_RED}; font-weight: bold'
        if val < 0:
            return f'color: {FT_RED}'
    return ''

styled = (crisis_df.style
    .format('{:.1f}%')
    .map(color_crisis)
)
style_returns_table(styled).set_caption('Returns During Crisis Periods (%)')

---
## Crisis Zoom Charts — 7 Periods, Indexed to 100

In [ ]:
n_crises = len(CRISES)
n_cols = 4
n_rows = math.ceil(n_crises / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
axes_flat = axes.flatten()

for idx, (label, start, end) in enumerate(CRISES):
    ax = axes_flat[idx]
    # Extend window slightly for context
    window_start = pd.Timestamp(start) - pd.Timedelta(days=30)
    window_end = pd.Timestamp(end) + pd.Timedelta(days=30)

    mask = (spy_norm.index >= window_start) & (spy_norm.index <= window_end)
    spy_slice = spy_norm[mask]
    if len(spy_slice) > 0:
        spy_rel = spy_slice / spy_slice.iloc[0] * 100
        ax.plot(spy_rel.index, spy_rel.values, 'k--', lw=2, label='SPY B&H')

    for r, c in zip(top5, TOP_COLORS):
        bal = pd.Series(r['balance_values'])
        bal.index = pd.to_datetime(bal.index)
        bal_mask = (bal.index >= window_start) & (bal.index <= window_end)
        bal_slice = bal[bal_mask]
        if len(bal_slice) > 0:
            bal_rel = bal_slice / bal_slice.iloc[0] * 100
            ax.plot(bal_rel.index, bal_rel.values, color=c, lw=1.5, alpha=0.85)

    ax.axhline(100, color=FT_GREY, ls=':', alpha=0.5)
    ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.12, color='#CC0000')
    ax.set_title(label, fontsize=11)
    if idx == 0:
        ax.legend(fontsize=6, loc='lower left')

# Hide extra subplots
for idx in range(n_crises, len(axes_flat)):
    axes_flat[idx].set_visible(False)

plt.suptitle('Crisis Period Zoom (indexed to 100)', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

---
## Monthly Return Distributions — Top 5

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 4), sharey=True)

for ax, r, c in zip(axes, top5, TOP_COLORS):
    bal = pd.Series(r['balance_values'])
    bal.index = pd.to_datetime(bal.index)
    # Monthly returns
    monthly = bal.resample('ME').last().pct_change().dropna() * 100

    ax.hist(monthly, bins=40, color=c, alpha=0.7, edgecolor='white')
    ax.axvline(monthly.mean(), color='black', ls='--', lw=1.5)
    ax.axvline(0, color=FT_GREY, ls=':', alpha=0.5)
    ax.set_title(r['name'], fontsize=8)
    ax.set_xlabel('Monthly Return %')

    skew = monthly.skew()
    kurt = monthly.kurtosis()
    ax.text(0.05, 0.95, f'skew={skew:.2f}\nkurt={kurt:.1f}\nmean={monthly.mean():.2f}%',
            transform=ax.transAxes, fontsize=7, va='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('Monthly Return Distributions — Top 5 Barbell Configs', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Conclusion

### What the sweep reveals

From ~924 configurations of the Taleb barbell (sell ATM straddle + buy deep OTM puts), with realistic margin constraints via `max_notional_pct`:

**ATM Straddle (sell side):**
- The ATM heatmaps show which DTE windows and exit timing harvest the Variance Risk Premium most efficiently
- Strike width matters: tighter strikes (±0.5%) concentrate theta decay but increase assignment risk
- Exit DTE interacts with the DTE window — exiting too early leaves premium on the table, too late risks gamma exposure

**Deep OTM Put (buy side):**
- Delta range is the key parameter — deeper OTM (< 0.05 delta) gives more convexity but costs more in bleed
- As Taleb noted: *"shorter DTE, deeper OTM"* tends to outperform *"longer DTE, higher delta"* for tail protection
- The OTM heatmap reveals the optimal delta/DTE tradeoff for crash insurance

**Notional Cap (`max_notional_pct`):**
- Without the cap, short straddles create 10-12x leverage, producing impossible drawdowns (-300%+)
- Cap at 30% of portfolio keeps max drawdown realistic (-60%) while preserving most of the premium income
- Phase 3 sweeps 20%, 30%, 50% caps — the risk/return tradeoff is visible in the scatter plot

**Combined (Phase 3):**
- The top configs in the combined sweep show how ATM income can fund OTM protection
- Allocation sizing matters enormously — the scatter plot shows the efficient frontier of barbell configs
- Crisis table reveals which configs actually protect during crashes vs which just reduce vol selling income

### The Taleb insight confirmed

The barbell is not about *making money* from OTM puts. It's about **reshaping the return distribution**:
- Pure ATM selling has negative skew (many small wins, rare catastrophic losses)
- Adding the OTM tail hedge truncates the left tail at the cost of some income
- The monthly distribution charts show whether the top configs achieve this truncation

### Key quotes from Taleb (AMA 2015)

> *"I am not against selling ATM premium."*

> *"ATM drops faster, OTM rises. The same idea of shadow theta."*

> *"For squeezes, a six month OTM (measuring in low delta) is preferable to a 1 year OTM with higher delta."*

### References

- Taleb, N.N. [Reddit AMA on Options (2015)](https://www.reddit.com/r/options/comments/38onec/)
- Taleb, N.N. *Dynamic Hedging* (Wiley, 1997)
- Spitznagel, M. *Safe Haven* (Wiley, 2021)
- Carr, P. & Wu, L. *Variance Risk Premia* (2009)
- Ilmanen, A. & Israelov, R. *Tail Risk Hedging* (AQR, 2018)